In [1]:
# Create Sagemaker execution role
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import boto3
import os

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()

s3_prefix = "script-mode-workflow"
pickle_s3_prefix = f"{s3_prefix}/pickle"
pickle_s3_uri = f"s3://{bucket}/{s3_prefix}/pickle"
pickle_train_s3_uri = f"{pickle_s3_uri}/train"

train_dir = os.path.join(os.getcwd(), "")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(pickle_s3_prefix, "train.pickle")).upload_file(
    train_dir + "/train.pickle"
)

In [3]:
# This is not required as these values are the defaults:
hyperparameters = {
    "copy_X": True,
    "fit_intercept": True,
    "normalize": False,
}

In [4]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Modify this based on your script name !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
entry_point = "regressionScript.py"

# Modify this based on your instance type / size
train_instance_type = "ml.m5.large"

estimator_parameters = {
    "entry_point": entry_point,
    "source_dir": "script",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "linearregression-model",
}

estimator = SKLearn(**estimator_parameters)

In [5]:
inputs = {
    "train": pickle_train_s3_uri
}

# starting the training job
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: linearregression-model-2024-11-09-15-57-39-418


2024-11-09 15:57:41 Starting - Starting the training job...
2024-11-09 15:57:55 Starting - Preparing the instances for training...
2024-11-09 15:58:41 Downloading - Downloading the training image......
2024-11-09 15:59:45 Training - Training image download completed. Training in progress.
2024-11-09 15:59:45 Uploading - Uploading generated training model
2024-11-09 15:59:45 Completed - Training job completed
2024-11-09 15:59:26,087 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-11-09 15:59:26,089 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-09 15:59:26,127 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-11-09 15:59:26,281 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-09 15:59:26,293 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-11-09 15:59:26,307 sagemaker-training-toolkit INFO 

In [6]:
sklearn_predictor = estimator.deploy(initial_instance_count=1,
                                     instance_type='ml.m5.large',
                                     endpoint_name='linearregression-endpoint')

INFO:sagemaker:Creating model with name: linearregression-model-2024-11-09-15-59-56-308
INFO:sagemaker:Creating endpoint-config with name linearregression-endpoint
INFO:sagemaker:Creating endpoint with name linearregression-endpoint


-------!

In [7]:
sklearn_predictor.predict([[0],[1],[2],[3]])

array([-1.01706818e-01,  7.17427592e+01,  1.43587225e+02,  2.15431691e+02])

In [8]:
sklearn_predictor.delete_endpoint(True)

INFO:sagemaker:Deleting endpoint configuration with name: linearregression-endpoint
INFO:sagemaker:Deleting endpoint with name: linearregression-endpoint
